In [1]:
import pyrealsense2 as rs
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
import copy
from IPython.display import clear_output
import time

np.set_printoptions(precision=4, suppress=True)

# Saving all frames 

In [2]:
# data filenames
d435_filename = 'data/D435.bag'
t265_filename = 'data/T265.bag'

### T265 (extract key frames, every 220th frame)

In [3]:
# Setup:
cfg = rs.config()
cfg.enable_device_from_file(t265_filename)
cfg.enable_stream(rs.stream.pose)
pipe = rs.pipeline()
profile = pipe.start(cfg)

t265_data_list = []
t265_time_list = []
first_timestamp = None
count = 0
while(True):

    frames = pipe.wait_for_frames()
    pose = frames.get_pose_frame()
    
    if pose:
        if pose.get_timestamp() == first_timestamp:
            print('reached the first frame - reading bag file completed')
            break
        if first_timestamp is None:
            first_timestamp = pose.get_timestamp()
            print('first_timestamp',first_timestamp)
            t265_data_list.append(pose.get_pose_data())
            t265_time_list.append(pose.get_timestamp())

        count+=1
        if count%220==0:
            print('current-first',pose.get_timestamp()-first_timestamp)
            t265_data_list.append(pose.get_pose_data())
            t265_time_list.append(pose.get_timestamp())
            
        
pipe.stop()

first_timestamp 1585059273040.0645
current-first 1094.823974609375
current-first 2194.82470703125
current-first 3294.497802734375
current-first 4394.272705078125
current-first 5494.173583984375
current-first 6594.005126953125
current-first 7693.84130859375
current-first 8793.719970703125
current-first 9893.547119140625
current-first 10993.33935546875
current-first 12093.226806640625
current-first 13193.03173828125
current-first 14292.933837890625
current-first 15392.739501953125
current-first 16492.5751953125
current-first 17592.388916015625
current-first 18692.2431640625
current-first 19792.046142578125
current-first 20891.94140625
current-first 21991.758056640625
current-first 23091.640625
current-first 24191.408203125
current-first 25291.20361328125
current-first 26391.15576171875
current-first 27490.9638671875
current-first 28590.75439453125
current-first 29690.663818359375
reached the first frame - reading bag file completed


In [4]:
len(t265_time_list)

28

In [5]:
number_of_key_frames = len(t265_time_list)

### Extract transformations from 265

In [6]:
def get_transformation(data):
#     data = pose.get_pose_data()
    data_rot = [float(i.strip('xyzw: ')) for i in str(data.rotation).split(', ')]
    r = R.from_quat(data_rot)
    rotation = np.array(r.as_matrix())
    translation = np.array([float(i.strip('xyzw: ')) for i in str(data.translation).split(', ')])[np.newaxis].T
    T = np.hstack((rotation, translation))
    T = np.vstack((T, np.array([0, 0, 0, 1])))
    return T

In [7]:
transformation_matrix_set = []
for pose in t265_data_list:
    transformation_matrix_set.append(get_transformation(pose))

### D435i iterate over all frames

In [28]:
# Setup:
cfg = rs.config()
cfg.enable_device_from_file(d435_filename)
cfg.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
pipe = rs.pipeline()
profile = pipe.start(cfg)

d435_data_list_all = []
d435_time_list_all = []
# a = np.array([])
# l = []
first_timestamp = None
counter = 0
while(True):

    frames = pipe.wait_for_frames()
    depth_frame = frames.get_depth_frame()
#     print('\ndepth_frame.get_timestamp()',depth_frame.get_timestamp())
    
    if depth_frame.get_timestamp() == first_timestamp:
        print('reached the first frame - reading bag file completed')
        print('timestamp',depth_frame.get_timestamp())
        break
    if first_timestamp is None:
        first_timestamp = depth_frame.get_timestamp()
        print('first_timestamp',first_timestamp)
    print('current-first',depth_frame.get_timestamp()-first_timestamp)
    
    d435_data_list_all.append(np.asanyarray(depth_frame.get_data()).copy())
    d435_time_list_all.append(depth_frame.get_timestamp())

    depth_image = np.asanyarray(depth_frame.get_data())
    depth_image = cv2.convertScaleAbs(depth_image, alpha=0.03)
    cv2.imshow('D435 Depth Frame', depth_image)
    cv2.waitKey(1)
    
cv2.destroyAllWindows()
pipe.stop()

first_timestamp 1585059272738.613
current-first 0.0
current-first 32.364013671875
current-first 65.6904296875
current-first 98.64453125
current-first 131.9052734375
current-first 165.260009765625
current-first 198.619384765625
current-first 231.972412109375
current-first 265.320556640625
current-first 298.669921875
current-first 332.087646484375
current-first 365.589111328125
current-first 398.846923828125
current-first 432.1474609375
current-first 465.772216796875
current-first 498.96435546875
current-first 532.234375
current-first 565.82763671875
current-first 598.99462890625
current-first 632.35302734375
current-first 665.62646484375
current-first 698.97509765625
current-first 732.40380859375
current-first 765.72802734375
current-first 799.047119140625
current-first 832.42724609375
current-first 865.82275390625
current-first 899.141357421875
current-first 932.950927734375
current-first 966.539306640625
current-first 999.246826171875
current-first 1032.65673828125
current-first 1065.

current-first 8937.414794921875
current-first 8970.74365234375
current-first 9004.131103515625
current-first 9037.5234375
current-first 9070.91943359375
current-first 9104.188720703125
current-first 9137.487548828125
current-first 9170.879638671875
current-first 9204.4130859375
current-first 9237.685546875
current-first 9270.93994140625
current-first 9304.329833984375
current-first 9337.723388671875
current-first 9371.021240234375
current-first 9404.494873046875
current-first 9437.6923828125
current-first 9471.15478515625
current-first 9504.37451171875
current-first 9537.784423828125
current-first 9571.24609375
current-first 9604.480224609375
current-first 9637.7802734375
current-first 9671.228515625
current-first 9704.627197265625
current-first 9737.8681640625
current-first 9771.18310546875
current-first 9804.661376953125
current-first 9838.06982421875
current-first 9871.497802734375
current-first 9904.677734375
current-first 9938.310546875
current-first 9971.3701171875
current-first 

current-first 17676.140380859375
current-first 17709.812744140625
current-first 17742.8505859375
current-first 17776.23046875
current-first 17809.529052734375
current-first 17843.444091796875
current-first 17877.308837890625
current-first 17909.65869140625
current-first 17942.952880859375
current-first 17976.346923828125
current-first 18009.655029296875
current-first 18043.059326171875
current-first 18076.3701171875
current-first 18109.759765625
current-first 18143.07958984375
current-first 18176.479736328125
current-first 18209.75830078125
current-first 18243.130126953125
current-first 18276.477294921875
current-first 18309.884765625
current-first 18343.191162109375
current-first 18376.5869140625
current-first 18410.087158203125
current-first 18443.265625
current-first 18476.666748046875
current-first 18510.0029296875
current-first 18543.320556640625
current-first 18577.076416015625
current-first 18610.040283203125
current-first 18643.38671875
current-first 18676.77783203125
current-f

current-first 26482.20703125
current-first 26514.983154296875
current-first 26548.885498046875
current-first 26581.6240234375
current-first 26615.031982421875
current-first 26648.266845703125
current-first 26681.665771484375
current-first 26714.9208984375
current-first 26748.472412109375
current-first 26781.6201171875
current-first 26815.01953125
current-first 26848.358642578125
current-first 26881.80029296875
current-first 26915.2099609375
current-first 26948.531005859375
current-first 26982.301513671875
current-first 27015.6650390625
current-first 27048.57421875
current-first 27081.874755859375
current-first 27115.19921875
current-first 27149.234619140625
current-first 27181.982666015625
current-first 27215.590576171875
current-first 27248.7890625
current-first 27281.9716796875
current-first 27315.276611328125
current-first 27348.65576171875
current-first 27381.997802734375
current-first 27415.37548828125
current-first 27448.746826171875
current-first 27483.710205078125
current-first

In [9]:
# find the correspondedces between t265_time_list (keyframes) and d435_time_list
d435_data_list = []
d435_time_list = []

for t265timestamp in t265_time_list:
    idx, val = min(enumerate(d435_time_list_all), key=lambda x: abs(x[1]-t265timestamp))
    d435_data_list.append(d435_data_list_all[idx])
    d435_time_list.append(d435_time_list_all[idx])

In [10]:
# just to check
i=27
t265_time_list[i], d435_time_list[i]

(1585059302730.7283, 1585059302722.2495)

In [11]:
len(d435_time_list)

28

### Save key frames from 435

In [12]:
# Setup:
cfg = rs.config()
cfg.enable_device_from_file(d435_filename)
cfg.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
pipe = rs.pipeline()
profile = pipe.start(cfg)

# d435_data_list_all = []
# d435_time_list_all = []
depth_frame_list = []
# a = np.array([])
# l = []
first_timestamp = None
counter = 0
while(True):

    frames = pipe.wait_for_frames()
    depth_frame = frames.get_depth_frame()
#     print('\ndepth_frame.get_timestamp()',depth_frame.get_timestamp())
    
    if depth_frame.get_timestamp() == first_timestamp:
        print('reached the first frame - reading bag file completed')
        print('timestamp',depth_frame.get_timestamp())
        break
    if first_timestamp is None:
        first_timestamp = depth_frame.get_timestamp()
        print('first_timestamp',first_timestamp)
#     print('current-first',depth_frame.get_timestamp()-first_timestamp)
    
    for d435timestamp in d435_time_list:
        if depth_frame.get_timestamp() == d435timestamp:
            depth_frame_list.append(depth_frame)
            print('current-first',depth_frame.get_timestamp()-first_timestamp)
    
#     d435_data_list_all.append(np.asanyarray(depth_frame.get_data()).copy())
#     d435_time_list_all.append(depth_frame.get_timestamp())

cv2.destroyAllWindows()
pipe.stop()

first_timestamp 1585059272738.613
current-first 298.669921875
current-first 1399.471923828125
current-first 2500.16064453125
current-first 3600.828369140625
current-first 4701.559326171875
current-first 5802.134521484375
current-first 6902.9052734375
current-first 8003.4609375
current-first 9104.188720703125
current-first 10204.84912109375
current-first 11306.476318359375
current-first 12406.26318359375
current-first 13506.83984375
current-first 14607.674560546875
current-first 15708.253662109375
current-first 16809.07177734375
current-first 17909.65869140625
current-first 18976.853271484375
current-first 20077.5517578125
current-first 21178.921630859375
current-first 22279.177734375
current-first 23380.199951171875
current-first 24481.905517578125
current-first 25581.21875
current-first 26681.665771484375
current-first 27782.395263671875
current-first 28882.953369140625
current-first 29983.636474609375
reached the first frame - reading bag file completed
timestamp 1585059272738.613


In [13]:
len(depth_frame_list)

28

# Transformations

transformation between cameras

$ {}^{T}T_D$ - d435 wrt t265, always the same

In [14]:
T_d_wrt_t = np.array([[0.999968402, -0.006753626, -0.004188075, -0.015890727],
                      [-0.006685408, -0.999848172, 0.016093893, 0.028273059],
                      [-0.004296131, -0.016065384, -0.999861654, -0.009375589],
                      [0, 0, 0, 1]])

In [15]:
def get_transformation(data):
#     data = pose.get_pose_data()
    data_rot = [float(i.strip('xyzw: ')) for i in str(data.rotation).split(', ')]
    r = R.from_quat(data_rot)
    rotation = np.array(r.as_matrix())
    translation = np.array([float(i.strip('xyzw: ')) for i in str(data.translation).split(', ')])[np.newaxis].T
    T = np.hstack((rotation, translation))
    T = np.vstack((T, np.array([0, 0, 0, 1])))
    return T

In [16]:
relative_transformation_matrix_set = [] # based on 265 = T10,T21,T32, ...
for i in range(len(transformation_matrix_set)-1):
    T_t1_wrt_w = transformation_matrix_set[i]
    T_t2_wrt_w = transformation_matrix_set[i+1]
    T_d1_wrt_w = T_t1_wrt_w @ T_d_wrt_t
    T_d2_wrt_w = T_t2_wrt_w @ T_d_wrt_t
    T_d2_wrt_d1 = np.linalg.inv(T_d1_wrt_w) @ T_d2_wrt_w
    relative_transformation_matrix_set.append(T_d2_wrt_d1)

In [17]:
len(transformation_matrix_set)

28

# Combine and plot PCs

http://www.open3d.org/docs/release/tutorial/Advanced/pointcloud_outlier_removal.html

In [18]:
def get_geom_pcl(depth_frame): #slower
    pc = rs.pointcloud()
    points = pc.calculate(depth_frame).as_points()
    coordinates = np.ndarray(buffer=points.get_vertices(), dtype=np.float32, shape=(480, 848, 3)) \
        .reshape((-1, 3))
    coordinates = coordinates[coordinates[:, 2] != 0]
    pcl = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(coordinates))
    return pcl

In [19]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [20]:
def downsample_and_inlier(pcl):
#     print("Downsample the point cloud with a voxel of 0.02")
    voxel_down_pcd = pcl.voxel_down_sample(voxel_size=0.02)
    # o3d.visualization.draw_geometries([voxel_down_pcd])

    # print("Every 5th points are selected")
    # uni_down_pcd = pcd.uniform_down_sample(every_k_points=5)
    # o3d.visualization.draw_geometries([uni_down_pcd])
    
#     print("Statistical oulier removal")
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=0.5)
    inlier_cloud1 = voxel_down_pcd.select_down_sample(ind)
    
#     print("Radius oulier removal")
    cl, ind = inlier_cloud1.remove_radius_outlier(nb_points=27, radius=0.05)
    inlier_cloud2 = inlier_cloud1.select_down_sample(ind)
    
    return inlier_cloud2

### Just plot two PCs

In [21]:
# for i in range(26):
i = 7
pcl1 = get_geom_pcl(depth_frame_list[i])
pcl2 = get_geom_pcl(depth_frame_list[i+1])
T = relative_transformation_matrix_set[i]

pcl1 = downsample_and_inlier(pcl1)
# o3d.visualization.draw_geometries([pcl1])

pcl2 = downsample_and_inlier(pcl2)
pcl2.transform(T)
# o3d.visualization.draw_geometries([pcl2])

o3d.visualization.draw_geometries([pcl1+pcl2])

# pcl_comb += copy.deepcopy(pcl2)
# o3d.visualization.draw_geometries([pcl_comb])

### Combine and plot all 28 PCs

In [22]:
pcl1 = get_geom_pcl(depth_frame_list[0])
pcl1 = downsample_and_inlier(pcl1)
pcl_comb = pcl1
print('PC# 0 fused')

for i in range(1,len(depth_frame_list)):
# i = 20
    pcl = get_geom_pcl(depth_frame_list[i])
    pcl = downsample_and_inlier(pcl)
    T = np.eye(4)
    for j in range(i):
        T = T @ relative_transformation_matrix_set[j]
    pcl.transform(T)
    pcl_comb+=pcl
    print('PC#',i,'fused')

o3d.visualization.draw_geometries([pcl_comb])

PC# 0 fused
PC# 1 fused
PC# 2 fused
PC# 3 fused
PC# 4 fused
PC# 5 fused
PC# 6 fused
PC# 7 fused
PC# 8 fused
PC# 9 fused
PC# 10 fused
PC# 11 fused
PC# 12 fused
PC# 13 fused
PC# 14 fused
PC# 15 fused
PC# 16 fused
PC# 17 fused
PC# 18 fused
PC# 19 fused
PC# 20 fused
PC# 21 fused
PC# 22 fused
PC# 23 fused
PC# 24 fused
PC# 25 fused
PC# 26 fused
PC# 27 fused


In [26]:
pcl_comb_opt = downsample_and_inlier(pcl_comb)
o3d.visualization.draw_geometries([pcl_comb_opt])

In [ ]:
1